# Handwriting to Text Recognition Project
Pranav Thiriveedhi

Dataset (Train, Test, Validation) from Kaggle

### Using Keras to build model that detects handwriting
Model: CRNN which is CNN + RNN

Trained using: CTC (Connectionist Temporal Classification) Loss


In [3]:
# Import libraries (python packages)
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# Import libraries (ML packages)
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Lambda, Activation, BatchNormalization, Dropout
from keras.optimizers import Adam

In [12]:
# Loading Train, Test and Validation data

train = pd.read_csv('/Users/pranavthiriveedhi1/Desktop/projects/Handwriting_Recognition/written_name_train_v2.csv')
test = pd.read_csv('/Users/pranavthiriveedhi1/Desktop/projects/Handwriting_Recognition/written_name_test_v2.csv')
validation = pd.read_csv('/Users/pranavthiriveedhi1/Desktop/projects/Handwriting_Recognition/written_name_validation_v2.csv')

In [17]:
# Cleaning data by removing NA values

print("Number of NaNs in train set      : ", train['IDENTITY'].isnull().sum())
print("Number of NaNs in validation set : ", validation['IDENTITY'].isnull().sum())

train.dropna(axis=0, inplace=True)
validation.dropna(axis=0, inplace=True)

Number of NaNs in train set      :  0
Number of NaNs in validation set :  0


In [19]:
train = train[train['IDENTITY'] != 'UNREADABLE']
validation = validation[validation['IDENTITY'] != 'UNREADABLE']

In [21]:
# Convert all labels to uppercase

train['IDENTITY'] = train['IDENTITY'].str.upper()
validation['IDENTITY'] = validation['IDENTITY'].str.upper()

In [23]:
train.reset_index(inplace = True, drop=True) 
validation.reset_index(inplace = True, drop=True)

In [25]:
# Preprocess images

def preprocess(img):
    (h, w) = img.shape
    final_img = np.ones([64, 256])*255 # blank white image
    # crop
    if w > 256:
        img = img[:, :256]  
    if h > 64:
        img = img[:64, :]    
    
    final_img[:h, :w] = img
    return cv2.rotate(final_img, cv2.ROTATE_90_CLOCKWISE)

In [26]:
train_size = 50000
valid_size = 5000

### Train 50,000 images and Validate 5,000 images

In [27]:
train_x = []

for i in range(train_size):
    img_dir = '/Users/pranavthiriveedhi1/Desktop/projects/Handwriting_Recognition/train/'+train.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image/255.
    train_x.append(image)

In [30]:
valid_x = []

for i in range(valid_size):
    img_dir = '/Users/pranavthiriveedhi1/Desktop/projects/Handwriting_Recognition/validation/'+validation.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image/255.
    valid_x.append(image)

In [31]:
# Reshape arrays
train_x = np.array(train_x).reshape(-1, 256, 64, 1)
valid_x = np.array(valid_x).reshape(-1, 256, 64, 1)

In [32]:
# Labeling for CTC Loss

alphabets = u"ABCDEFGHIJKLMNOPQRSTUVWXYZ-' "
max_str_len = 24 # max length of input labels
num_of_characters = len(alphabets) + 1 # +1 for ctc pseudo blank
num_of_timestamps = 64 # max length of predicted labels


In [33]:
def label_to_num(label):
    label_num = []
    for ch in label:
        label_num.append(alphabets.find(ch))
        
    return np.array(label_num)


In [34]:
def num_to_label(num):
    ret = ""
    for ch in num:
        if ch == -1:  # CTC Blank
            break
        else:
            ret+=alphabets[ch]
    return ret

In [35]:
name = 'PRANAV'
print(name, '\n',label_to_num(name))

PRANAV 
 [15 17  0 13  0 21]


In [45]:
train_y = np.ones([train_size, max_str_len]) * -1
train_label_len = np.zeros([train_size, 1])
train_input_len = np.ones([train_size, 1]) * (num_of_timestamps-2)
train_output = np.zeros([train_size])  

valid_y = np.ones([valid_size, max_str_len]) * -1
valid_label_len = np.zeros([valid_size, 1])
valid_input_len = np.ones([valid_size, 1]) * (num_of_timestamps-2)
valid_output = np.zeros([valid_size])

for i in range(valid_size):
    valid_label_len[i] = len(validation.loc[i, 'IDENTITY'])
    valid_y[i, 0:len(validation.loc[i, 'IDENTITY'])]= label_to_num(validation.loc[i, 'IDENTITY'])   

In [46]:
print('True label : ',train.loc[100, 'IDENTITY'] , '\ntrain_y : ',train_y[100],'\ntrain_label_len : ',train_label_len[100], 
      '\ntrain_input_len : ', train_input_len[100])

True label :  NOUR 
train_y :  [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.] 
train_label_len :  [0.] 
train_input_len :  [62.]


In [47]:
input_data = Input(shape=(256, 64, 1), name='input')

inner = Conv2D(32, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(input_data)  
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)

inner = Conv2D(64, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(inner)
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
inner = Dropout(0.3)(inner)

inner = Conv2D(128, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(inner)
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(1, 2), name='max3')(inner)
inner = Dropout(0.3)(inner)

In [48]:
# CNN to RNN
inner = Reshape(target_shape=((64, 1024)), name='reshape')(inner)
inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)

## RNN
inner = Bidirectional(LSTM(256, return_sequences=True), name = 'lstm1')(inner)
inner = Bidirectional(LSTM(256, return_sequences=True), name = 'lstm2')(inner)

## OUTPUT
inner = Dense(num_of_characters, kernel_initializer='he_normal',name='dense2')(inner)
y_pred = Activation('softmax', name='softmax')(inner)

model = Model(inputs=input_data, outputs=y_pred)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 256, 64, 1)]      0         
                                                                 
 conv1 (Conv2D)              (None, 256, 64, 32)       320       
                                                                 
 batch_normalization (BatchN  (None, 256, 64, 32)      128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 256, 64, 32)       0         
                                                                 
 max1 (MaxPooling2D)         (None, 128, 32, 32)       0         
                                                                 
 conv2 (Conv2D)              (None, 128, 32, 64)       18496     
                                                             

In [49]:
# the ctc loss function
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [50]:
labels = Input(name='gtruth_labels', shape=[max_str_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

ctc_loss = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
model_final = Model(inputs=[input_data, labels, input_length, label_length], outputs=ctc_loss)

In [52]:
preds = model.predict(valid_x)
decoded = K.get_value(K.ctc_decode(preds, input_length=np.ones(preds.shape[0])*preds.shape[1], 
                                   greedy=True)[0][0])

prediction = []
for i in range(valid_size):
    prediction.append(num_to_label(decoded[i]))

157/157 [==============================] - 39s 237ms/step


In [54]:
y_true = validation.loc[0:valid_size, 'IDENTITY']
correct_char = 0
total_char = 0
correct = 0

for i in range(valid_size):
    pr = prediction[i]
    tr = y_true[i]
    total_char += len(tr)
    
    for j in range(min(len(tr), len(pr))):
        if tr[j] == pr[j]:
            correct_char += 1
            
    if pr == tr :
        correct += 1 
    
print('Correct characters predicted : %.2f%%' %(correct_char*100/total_char))
print('Correct words predicted      : %.2f%%' %(correct*100/valid_size))

Correct characters predicted : 3.72%
Correct words predicted      : 0.00%
